In [1]:
import logging
from tqdm import tqdm
import sys
sys.path.append('.')
from pathlib import Path
import pandas as pd
from cbastats.DBHelper import MongoDBHelper
import os
from dotenv import load_dotenv
from utils.datarefresher import get_current_season

In [2]:
config={}

In [3]:
current_season_year = int(get_current_season())
config['CURRENT_SEASON'] = f"{current_season_year-1}-{current_season_year}"

In [4]:
config['COLUMN_NAMES'] = ['season', 'DATE', 'boxscores_url', 'game_id', 'HOME', 'VISITOR',
       'HOME_PTS', 'VISITOR_PTS', 'Pace_home', 'eFG%_home', 'TOV%_home',
       'TS%_home', '3PAr_home', 'FTr_home', 'DRB%_home', 'TRB%_home',
       'AST%_home', 'STL%_home', 'BLK%_home', 'DRtg_home', 'ORB%_home',
       'FT/FGA_home', 'ORtg_home', 'Pace_visitor', 'eFG%_visitor',
       'TOV%_visitor', 'ORB%_visitor', 'FT/FGA_visitor', 'ORtg_visitor',
       'TS%_visitor', '3PAr_visitor', 'FTr_visitor', 'DRB%_visitor',
       'TRB%_visitor', 'AST%_visitor', 'STL%_visitor', 'BLK%_visitor',
       'DRtg_visitor', 'season_nth_game', 'hometeam_nth_homegame',
       'visitorteam_nth_visitorgame', 'TOTAL_PTS', 'HOME_VISITOR_PTS_DIFF',
       'HOME_WIN']
config['NUMERIC_COLS'] = ['VISITOR_PTS','HOME_PTS','Pace_home', 'eFG%_home', 'TOV%_home',
       'ORB%_home', 'FT/FGA_home', 'ORtg_home', 'Pace_visitor', 'eFG%_visitor',
       'TOV%_visitor', 'ORB%_visitor', 'FT/FGA_visitor', 'ORtg_visitor',
       'TS%_home', '3PAr_home', 'FTr_home', 'DRB%_home', 'TRB%_home',
       'AST%_home', 'STL%_home', 'BLK%_home', 'DRtg_home', 'TS%_visitor',
       '3PAr_visitor', 'FTr_visitor', 'DRB%_visitor', 'TRB%_visitor',
       'AST%_visitor', 'STL%_visitor', 'BLK%_visitor', 'DRtg_visitor']
config['DUP_COL'] = ['PLAYER','MP','eFG%','TOV%','USG%','ORB%','ORtg','BPM','Unnamed: 16_level_1']

In [5]:
DOTENV_PATH="."
env_path = Path(DOTENV_PATH) / '.env'
if not (env_path.exists()):
    print('.env file is missing.')
    sys.exit()
load_dotenv(dotenv_path=env_path,verbose=True)

True

In [6]:
needed_envs = ['MONGODB_PWD', 'MONGODB_USERNAME', 'MONGODB_ENDPOINT']
envs = os.environ
# only checks if user wants to save data to DB
# check if all needed environment variables are present

for needed_env in needed_envs:
    if needed_env not in envs:
        raise Exception(f"Missing environment variable: {needed_env}.\n \
    Please check if these environment variables are present: {needed_envs}")
    config[needed_env] = envs[needed_env]

In [7]:
mongodbio = MongoDBHelper()
client = mongodbio.create_connection(
    config['MONGODB_USERNAME'], config['MONGODB_PWD'], config['MONGODB_ENDPOINT'])
nba_db = client['nbaStats']
coll_nbaGames = nba_db['nbaGames']
coll_nbaGamesStaging= nba_db['nbaGamesStaging']

existing database ['cbaStats', 'nbaStats', 'admin', 'local']


# Data Cleaning Test

## Assemble Raw Data

In [8]:
# schedule
schedule = mongodbio.select_records(coll_nbaGames,field={
    'four_factors':0, 'basic_boxscores':0,'advanced_boxscores':0,'_id':0
})
schedule=pd.DataFrame(schedule)
print(schedule.shape)
schedule.head()

(9162, 8)


,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,season,boxscores_url,game_id
0,2020-12-22,GSW,99.0,BRK,125.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012220BRK
1,2020-12-22,LAC,116.0,LAL,109.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012220LAL
2,2020-12-23,CHO,114.0,CLE,121.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012230CLE
3,2020-12-23,NYK,107.0,IND,121.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012230IND
4,2020-12-23,MIA,107.0,ORL,113.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012230ORL


In [9]:
# four factors
fourfactors=mongodbio.select_records(coll_nbaGames,field={'four_factors':1,'_id':0
})

fourfactors_dfs=[]
for ff in fourfactors:
    _ = pd.DataFrame(ff['four_factors'])
    fourfactors_dfs.append(_)
fourfactors = pd.concat(fourfactors_dfs)
fourfactors = fourfactors.reset_index(drop=True)
print(fourfactors.shape)
fourfactors.head()

(18324, 8)


,Team,Pace,eFG%,TOV%,ORB%,FT/FGA,ORtg,game_id
0,GSW,110.5,0.424,14.2,22.8,0.152,89.6,202012220BRK
1,BRK,110.5,0.538,15.9,27.7,0.283,113.1,202012220BRK
2,LAC,103.5,0.548,13.6,22.9,0.151,112.1,202012220LAL
3,LAL,103.5,0.525,16.7,21.6,0.296,105.3,202012220LAL
4,CHO,101.5,0.589,13.4,16.7,0.089,112.3,202012230CLE


In [10]:
# box scores
adv_box_scores = mongodbio.select_records(coll_nbaGames,field={
    'game_id':1,'advanced_boxscores':1,'_id':0
})

team_adv_box_scores_dfs = []
bad_games=[]
for adv_box_score in adv_box_scores:
    game_id = adv_box_score['game_id']
    team_adv_box_score = adv_box_score['advanced_boxscores']
    for team, team_adv_box_score in team_adv_box_score.items():
        team_adv_box_score = pd.DataFrame(team_adv_box_score)
        team_adv_box_score['Team']=team
        team_adv_box_score['game_id']=game_id
        team_adv_box_scores_dfs.append(team_adv_box_score.iloc[-1])

adv_box_scores=pd.concat(team_adv_box_scores_dfs,axis=1).transpose().reset_index(drop=True)

# delete duplicated columns and empty, useless columns
rm_cols = config['DUP_COL']
adv_box_scores = adv_box_scores.drop(columns=rm_cols)
print(adv_box_scores.shape)
adv_box_scores.head()

(18324, 11)


,TS%,3PAr,FTr,DRB%,TRB%,AST%,STL%,BLK%,DRtg,Team,game_id
0,.589,.380,.348,77.2,54.8,57.1,10.0,10.6,89.6,BRK,202012220BRK
1,.454,.333,.232,72.3,45.2,70.3,5.4,10.5,113.1,GSW,202012220BRK
2,.576,.358,.383,77.1,52.9,57.9,3.9,3.8,112.1,LAL,202012220LAL
3,.572,.430,.204,78.4,47.1,50.0,9.7,5.8,105.3,LAC,202012220LAL
4,.632,.345,.230,83.3,61.0,73.9,11.8,6.5,112.3,CLE,202012230CLE


## Join Data Together

In [11]:
# join with four factors
_ = pd.merge(schedule,fourfactors,left_on=['game_id','HOME'],right_on=['game_id','Team'])
_ = pd.merge(_,fourfactors,left_on=['game_id','VISITOR'],right_on=['game_id','Team'],suffixes=('_home','_visitor'))
# join with box scores
_ = pd.merge(_,adv_box_scores,left_on=['game_id','Team_home'],right_on=['game_id','Team'])
complete_data = pd.merge(_,adv_box_scores,left_on=['game_id','Team_visitor'],right_on=['game_id','Team'],suffixes=('_home','_visitor'))
complete_data = complete_data.drop(columns=['Team_home','Team_visitor'])

In [12]:
complete_data

,DATE,VISITOR,VISITOR_PTS,HOME,HOME_PTS,season,boxscores_url,game_id,Pace_home,eFG%_home,...,DRtg_home,TS%_visitor,3PAr_visitor,FTr_visitor,DRB%_visitor,TRB%_visitor,AST%_visitor,STL%_visitor,BLK%_visitor,DRtg_visitor
0,2020-12-22,GSW,99.0,BRK,125.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012220BRK,110.5,0.538,...,89.6,.454,.333,.232,72.3,45.2,70.3,5.4,10.5,113.1
1,2020-12-22,LAC,116.0,LAL,109.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012220LAL,103.5,0.525,...,112.1,.572,.430,.204,78.4,47.1,50.0,9.7,5.8,105.3
2,2020-12-23,CHO,114.0,CLE,121.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012230CLE,101.5,0.609,...,112.3,.590,.489,.167,70.6,39.0,64.4,9.8,7.0,119.2
3,2020-12-23,NYK,107.0,IND,121.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012230IND,106.0,0.532,...,101.0,.562,.329,.271,77.8,44.4,64.1,7.6,13.3,114.2
4,2020-12-23,MIA,107.0,ORL,113.0,2020-2021,https://www.basketball-reference.com/boxscores...,202012230ORL,104.9,0.534,...,102.0,.586,.241,.229,76.6,51.2,59.5,10.5,3.3,107.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9157,2021-04-02,ATL,126.0,NOP,103.0,2020-2021,https://www.basketball-reference.com/boxscores...,202104020NOP,104.6,0.477,...,120.5,.655,.333,.241,87.8,55.3,51.0,9.6,15.3,98.5
9158,2021-04-02,CHI,106.0,UTA,113.0,2020-2021,https://www.basketball-reference.com/boxscores...,202104020UTA,93.5,0.551,...,113.4,.549,.315,.191,85.0,48.8,60.5,5.3,7.0,120.8
9159,2021-04-02,OKC,103.0,PHO,140.0,2020-2021,https://www.basketball-reference.com/boxscores...,202104020PHO,100.9,0.683,...,102.1,.520,.427,.258,81.8,48.1,54.3,5.9,5.3,138.8
9160,2021-04-02,MIL,127.0,POR,109.0,2020-2021,https://www.basketball-reference.com/boxscores...,202104020POR,106.8,0.470,...,118.9,.646,.433,.211,78.0,52.6,51.0,6.6,11.1,102.0


In [13]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9162 entries, 0 to 9161
Data columns (total 38 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   DATE            9162 non-null   datetime64[ns]
 1   VISITOR         9162 non-null   object        
 2   VISITOR_PTS     9162 non-null   object        
 3   HOME            9162 non-null   object        
 4   HOME_PTS        9162 non-null   object        
 5   season          9162 non-null   object        
 6   boxscores_url   9162 non-null   object        
 7   game_id         9162 non-null   object        
 8   Pace_home       9162 non-null   float64       
 9   eFG%_home       9162 non-null   float64       
 10  TOV%_home       9162 non-null   float64       
 11  ORB%_home       9162 non-null   float64       
 12  FT/FGA_home     9162 non-null   float64       
 13  ORtg_home       9162 non-null   float64       
 14  Pace_visitor    9162 non-null   float64       
 15  eFG%

# Convert Data Types

In [14]:
complete_data[config['NUMERIC_COLS']] = complete_data[config['NUMERIC_COLS']].astype('float')

# Add Aux Columns

In [15]:
complete_data['season_nth_game'] = complete_data.groupby(['season']).cumcount()+1
complete_data['hometeam_nth_homegame'] = complete_data.groupby(['season','HOME']).cumcount()+1
complete_data['visitorteam_nth_visitorgame'] = complete_data.groupby(['season','VISITOR']).cumcount()+1

complete_data['TOTAL_PTS'] = complete_data['VISITOR_PTS']+complete_data['HOME_PTS']
complete_data['HOME_VISITOR_PTS_DIFF'] = complete_data['HOME_PTS']-complete_data['VISITOR_PTS']
complete_data['HOME_WIN'] = (complete_data['HOME_VISITOR_PTS_DIFF']>0)*1

In [16]:
cols_diff=set(complete_data.columns).symmetric_difference(set(config['COLUMN_NAMES']))
assert len(cols_diff)==0, f"Difference in columns: {cols_diff}"

In [17]:
# rearrange column orders
complete_data = complete_data[config['COLUMN_NAMES']]

In [20]:
coll_nbaGameBoxScores = nba_db['nbaProcessedBoxScores']

coll_nbaGameBoxScores.delete_many(filter={})

coll_nbaGameBoxScores.insert_many(complete_data.to_dict('records'))